In [4]:
# Environment Setup and Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from torch.cuda.amp import autocast, GradScaler

import os
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt
from pathlib import Path
import json
import time
from tqdm import tqdm
import gc
import warnings
warnings.filterwarnings('ignore')

# Dask for distributed processing
import dask
from dask import delayed
from dask.distributed import Client
import multiprocessing as mp

print("Environment setup complete!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f}GB")

Environment setup complete!
PyTorch version: 2.7.1+cu118
CUDA available: True
GPU: NVIDIA GeForce RTX 4060 Laptop GPU
GPU Memory: 8.0GB


In [2]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Model architecture classes
class DoubleConv(nn.Module):
    """Double convolution block with BatchNorm and ReLU"""
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)

class Down(nn.Module):
    """Downsampling block with maxpool followed by double conv"""
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)

class Up(nn.Module):
    """Upsampling block with transpose conv and skip connections"""
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.up = nn.ConvTranspose2d(in_channels, in_channels // 2, kernel_size=2, stride=2)
        self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        
        # Handle Size Mismatch
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]
        x1 = nn.functional.pad(x1, [diffX // 2, diffX - diffX // 2,
                                    diffY // 2, diffY - diffY // 2])
        
        # Concatenate skip connection
        x = torch.cat([x2, x1], dim=1)
        
        return self.conv(x)

class OutConv(nn.Module):
    """Final output convolution"""
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)

class UNetWithHeight(nn.Module):
    """Enhanced U-Net with height estimation capabilities"""
    def __init__(self, n_channels, n_classes):
        super(UNetWithHeight, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes

        # Encoder
        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        self.down4 = Down(512, 1024)
        
        # Decoder for segmentation
        self.up1 = Up(1024, 512)
        self.up2 = Up(512, 256)
        self.up3 = Up(256, 128)
        self.up4 = Up(128, 64)
        self.outc = OutConv(64, n_classes)
        
        # Height estimation branch
        self.height_up1 = Up(1024, 512)
        self.height_up2 = Up(512, 256)
        self.height_up3 = Up(256, 128)
        self.height_up4 = Up(128, 64)
        self.height_out = OutConv(64, 1)  # Single channel for height

    def forward(self, x):
        # Encoder path
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        
        # Segmentation decoder path
        seg_x = self.up1(x5, x4)
        seg_x = self.up2(seg_x, x3)
        seg_x = self.up3(seg_x, x2)
        seg_x = self.up4(seg_x, x1)
        segmentation = self.outc(seg_x)
        
        # Height estimation decoder path
        height_x = self.height_up1(x5, x4)
        height_x = self.height_up2(height_x, x3)
        height_x = self.height_up3(height_x, x2)
        height_x = self.height_up4(height_x, x1)
        height_map = self.height_out(height_x)
        
        return segmentation, height_map

# Initialize the correct model architecture
model = UNetWithHeight(n_channels=3, n_classes=1).to(device)
print(f"Model initialized with parameters: {sum(p.numel() for p in model.parameters()):,}")

def load_model_weights(model, checkpoint_path):
    """Load model weights from checkpoint"""
    if not os.path.exists(checkpoint_path):
        print(f"Warning: Checkpoint {checkpoint_path} not found. Using random weights.")
        return False
    
    print(f"Loading weights from {checkpoint_path}")
    checkpoint = torch.load(checkpoint_path, map_location=device, weights_only=False)
    if 'model_state_dict' in checkpoint:
        model.load_state_dict(checkpoint['model_state_dict'])
    else:
        model.load_state_dict(checkpoint)
    
    model.eval()
    print("Model weights loaded successfully!")
    return True

model_loaded = load_model_weights(model, "height_analysis_output/best_height_model.pth")
print("Model ready for large-scale processing!")

Using device: cuda
Model initialized with parameters: 43,228,098
Loading weights from height_analysis_output/best_height_model.pth
Model weights loaded successfully!
Model ready for large-scale processing!


In [3]:
# Large-Scale Dataset for 500km x 500km Areas
class LargeScaleDataset(Dataset):
    """Dataset designed for processing very large satellite imagery (500km x 500km)"""
    
    def __init__(self, image_paths, patch_size=512, overlap=0.1, transform=None):
        self.image_paths = image_paths
        self.patch_size = patch_size
        self.overlap = overlap
        self.transform = transform
        self.patches_info = []
        self._calculate_patches()
    
    def _calculate_patches(self):
        """Calculate all patches for efficient large-scale processing"""
        step = int(self.patch_size * (1 - self.overlap))
        
        for img_idx, img_path in enumerate(self.image_paths):
            if not Path(img_path).exists():
                print(f"Warning: Image {img_path} not found, creating dummy entry")
                # Create dummy large image dimensions for 500km x 500km
                h, w = 50000, 50000  # Very large image simulation
            else:
                img = Image.open(img_path)
                w, h = img.size
            
            # Calculate patches for this large image
            for y in range(0, h - self.patch_size + 1, step):
                for x in range(0, w - self.patch_size + 1, step):
                    self.patches_info.append({
                        'img_idx': img_idx,
                        'img_path': img_path,
                        'x': x,
                        'y': y,
                        'patch_size': self.patch_size
                    })
    
    def __len__(self):
        return len(self.patches_info)
    
    def __getitem__(self, idx):
        patch_info = self.patches_info[idx]
        
        # Load only the required patch (memory efficient for large images)
        if Path(patch_info['img_path']).exists():
            img = np.array(Image.open(patch_info['img_path']))
            x, y = patch_info['x'], patch_info['y']
            patch = img[y:y+self.patch_size, x:x+self.patch_size]
        else:
            # Dummy patch for testing
            patch = np.random.randint(0, 255, (self.patch_size, self.patch_size, 3), dtype=np.uint8)
        
        if self.transform:
            patch = self.transform(patch)
        else:
            patch = torch.FloatTensor(patch).permute(2, 0, 1) / 255.0
        
        return patch, patch_info

# Create transforms and test dataset
large_scale_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Test with dummy large area paths
test_large_area_paths = ["large_area_1.tif", "large_area_2.tif"]

dataset = LargeScaleDataset(
    test_large_area_paths, 
    patch_size=512, 
    overlap=0.1, 
    transform=large_scale_transforms
)

print(f"Created large-scale dataset with {len(dataset)} patches")
print(f"Dataset ready for 500km x 500km processing!")

Created large-scale dataset with 23328 patches
Dataset ready for 500km x 500km processing!


In [4]:
# Memory-Efficient Inference for Large Areas
def memory_efficient_large_scale_inference(model, dataloader, device, use_mixed_precision=True):
    """Memory-efficient inference optimized for processing 500km x 500km areas"""
    model.eval()
    predictions = []
    patch_infos = []
    
    # Clear GPU cache before processing
    torch.cuda.empty_cache()
    
    # Use automatic mixed precision for memory efficiency
    scaler = GradScaler() if use_mixed_precision else None
    
    with torch.no_grad():
        pbar = tqdm(dataloader, desc="Large-scale inference (500km x 500km)")
        
        for batch_idx, (images, batch_patch_infos) in enumerate(pbar):
            images = images.to(device, non_blocking=True)
            
            # Use mixed precision to save memory
            if use_mixed_precision:
                with autocast():
                    outputs = model(images)
                    # Handle dual output from UNetWithHeight (segmentation, height_map)
                    if isinstance(outputs, tuple):
                        segmentation, height_map = outputs
                        # Use segmentation for building detection
                        outputs = torch.sigmoid(segmentation)
                    else:
                        outputs = torch.sigmoid(outputs)
            else:
                outputs = model(images)
                # Handle dual output from UNetWithHeight (segmentation, height_map)
                if isinstance(outputs, tuple):
                    segmentation, height_map = outputs
                    # Use segmentation for building detection
                    outputs = torch.sigmoid(segmentation)
                else:
                    outputs = torch.sigmoid(outputs)
            
            # Move to CPU immediately to free GPU memory
            batch_predictions = outputs.cpu().numpy()
            predictions.extend(batch_predictions)
            patch_infos.extend(batch_patch_infos)
            
            # Memory management for large-scale processing
            if batch_idx % 10 == 0:  # Clear cache every 10 batches
                torch.cuda.empty_cache()
                
            # Update progress bar with memory info
            if torch.cuda.is_available():
                memory_used = torch.cuda.memory_allocated() / 1024**3
                pbar.set_postfix({'GPU_Memory_GB': f'{memory_used:.1f}'})
    
    return predictions, patch_infos

# Test memory-efficient inference
if model_loaded:
    # Create test dataloader
    test_dataloader = DataLoader(
        dataset, 
        batch_size=2, 
        shuffle=False, 
        num_workers=0
    )
    
    # Test with small subset
    small_subset = torch.utils.data.Subset(dataset, range(min(10, len(dataset))))
    small_dataloader = DataLoader(small_subset, batch_size=2, shuffle=False)
    
    test_predictions, test_patch_infos = memory_efficient_large_scale_inference(
        model, small_dataloader, device
    )
    
    print(f"Memory-efficient inference test completed!")
    print(f"Generated {len(test_predictions)} predictions efficiently")
    print(f"Using UNetWithHeight model with dual outputs (segmentation + height)")
    print(f"Dummy data patches working correctly for testing!")
else:
    print("Skipping inference test - model not loaded")

Large-scale inference (500km x 500km): 100%|██████████| 5/5 [00:01<00:00,  4.93it/s, GPU_Memory_GB=0.2]

Memory-efficient inference test completed!
Generated 10 predictions efficiently
Using UNetWithHeight model with dual outputs (segmentation + height)
Dummy data patches working correctly for testing!


In [5]:
# Tiling Strategy for 500km x 500km Areas
class LargeAreaTilingStrategy:
    """Advanced tiling strategy specifically designed for 500km x 500km areas"""
    
    def __init__(self, patch_size=512, overlap=0.1, max_memory_gb=4):
        self.patch_size = patch_size
        self.overlap = overlap
        self.max_memory_gb = max_memory_gb
        self.step = int(patch_size * (1 - overlap))
    
    def calculate_optimal_batch_size_for_large_areas(self, estimated_image_size_gb):
        """Calculate optimal batch size for very large satellite imagery"""
        # Conservative memory estimation for 500km x 500km processing
        patch_memory_mb = (self.patch_size * self.patch_size * 3 * 4) / (1024 * 1024)
        
        # Reserve memory for model and other operations
        available_memory_mb = (self.max_memory_gb * 1024) - 1000  # Reserve 1GB
        
        # Calculate safe batch size
        optimal_batch_size = max(1, int(available_memory_mb / (patch_memory_mb * 3)))  # Factor of 3 for safety
        
        return min(optimal_batch_size, 4)  # Cap for large-area processing
    
    def create_large_area_tiling_plan(self, image_shape):
        """Create tiling plan optimized for 500km x 500km areas"""
        h, w = image_shape[:2]
        
        # Calculate total number of tiles for large area
        tiles_x = (w - self.patch_size) // self.step + 1
        tiles_y = (h - self.patch_size) // self.step + 1
        total_tiles = tiles_x * tiles_y
        
        print(f"Large area tiling plan: {tiles_x} x {tiles_y} = {total_tiles} tiles")
        print(f"Estimated processing time: {total_tiles * 0.1:.1f} seconds")
        
        tiles = []
        for y in range(0, h - self.patch_size + 1, self.step):
            for x in range(0, w - self.patch_size + 1, self.step):
                tiles.append({
                    'x': x,
                    'y': y,
                    'x_end': min(x + self.patch_size, w),
                    'y_end': min(y + self.patch_size, h)
                })
        
        return tiles
    
    def reconstruct_large_area_from_tiles(self, predictions, tiles, original_shape):
        """Reconstruct 500km x 500km area from overlapping tiles with blending"""
        h, w = original_shape[:2]
        
        # Initialize output arrays
        output = np.zeros((h, w), dtype=np.float32)
        weight_map = np.zeros((h, w), dtype=np.float32)
        
        # Create blending weights for seamless large-area reconstruction
        blend_weights = self._create_large_area_blend_weights(self.patch_size)
        
        print(f"Reconstructing large area: {h} x {w} pixels from {len(predictions)} tiles")
        
        for pred, tile in zip(predictions, tiles):
            if len(pred.shape) == 3:
                pred_tile = pred[0]  # Remove batch dimension
            else:
                pred_tile = pred
            
            x, y = tile['x'], tile['y']
            x_end, y_end = tile['x_end'], tile['y_end']
            
            # Handle edge cases for large areas
            tile_h, tile_w = pred_tile.shape
            actual_h = min(tile_h, y_end - y)
            actual_w = min(tile_w, x_end - x)
            
            # Apply blending weights
            weighted_pred = pred_tile[:actual_h, :actual_w] * blend_weights[:actual_h, :actual_w]
            weight_tile = blend_weights[:actual_h, :actual_w]
            
            # Accumulate in output
            output[y:y+actual_h, x:x+actual_w] += weighted_pred
            weight_map[y:y+actual_h, x:x+actual_w] += weight_tile
        
        # Normalize by weights
        output = np.divide(output, weight_map, out=np.zeros_like(output), where=weight_map!=0)
        
        return output
    
    def _create_large_area_blend_weights(self, patch_size):
        """Create blending weights optimized for large-area seamless reconstruction"""
        # Create smoother blending for large areas
        center = patch_size // 2
        y, x = np.ogrid[:patch_size, :patch_size]
        
        # Distance from center with smoother falloff
        dist_from_center = np.sqrt((x - center)**2 + (y - center)**2)
        max_dist = np.sqrt(2) * center
        
        # Smoother weights for large-area processing
        weights = np.cos(np.pi * dist_from_center / (2 * max_dist))
        weights = np.clip(weights, 0.2, 1.0)  # Ensure minimum weight
        
        return weights

# Test tiling strategy
large_area_tiling = LargeAreaTilingStrategy(patch_size=512, overlap=0.1, max_memory_gb=4)

# Test with 500km x 500km simulation
large_image_shape = (50000, 50000, 3)  # 500km x 500km at 10m resolution
tiles = large_area_tiling.create_large_area_tiling_plan(large_image_shape)
optimal_batch_size = large_area_tiling.calculate_optimal_batch_size_for_large_areas(estimated_image_size_gb=7.5)

print(f"Tiling strategy test completed:")
print(f"  Image shape: {large_image_shape}")
print(f"  Number of tiles: {len(tiles)}")
print(f"  Optimal batch size: {optimal_batch_size}")

Large area tiling plan: 108 x 108 = 11664 tiles
Estimated processing time: 1166.4 seconds
Tiling strategy test completed:
  Image shape: (50000, 50000, 3)
  Number of tiles: 11664
  Optimal batch size: 4


In [6]:
# Scene Mosaicking and Seamless Integration
def seamless_large_area_mosaicking(tile_predictions, tile_positions, output_shape, blend_border=64):
    """Advanced mosaicking for seamless integration of 500km x 500km areas"""
    
    h, w = output_shape[:2]
    mosaic = np.zeros((h, w), dtype=np.float32)
    weight_map = np.zeros((h, w), dtype=np.float32)
    
    print(f"Creating seamless mosaic for large area: {h} x {w} pixels")
    
    def create_advanced_feather_weights(tile_shape, border_size):
        """Create advanced feathering weights for large-area seamless blending"""
        th, tw = tile_shape
        weights = np.ones((th, tw), dtype=np.float32)
        
        # Apply sophisticated feathering to edges
        for i in range(border_size):
            alpha = (i + 1) / border_size
            # Smooth transition using cosine function
            weight = 0.5 * (1 + np.cos(np.pi * (1 - alpha)))
            
            # Apply to all edges
            weights[i, :] *= weight  # Top
            weights[-(i+1), :] *= weight  # Bottom
            weights[:, i] *= weight  # Left
            weights[:, -(i+1)] *= weight  # Right
        
        return weights
    
    # Process each tile with advanced blending
    for prediction, position in tqdm(zip(tile_predictions, tile_positions), 
                                   desc="Mosaicking large area", 
                                   total=len(tile_predictions)):
        x, y = position['x'], position['y']
        
        # Get tile dimensions
        if len(prediction.shape) == 3:
            pred_tile = prediction[0]
        else:
            pred_tile = prediction
        
        th, tw = pred_tile.shape
        
        # Create advanced feathering weights
        feather_weights = create_advanced_feather_weights((th, tw), blend_border)
        
        # Calculate placement area (handle large image boundaries)
        x_end = min(x + tw, w)
        y_end = min(y + th, h)
        actual_tw = x_end - x
        actual_th = y_end - y
        
        # Place tile in mosaic with advanced blending
        mosaic[y:y_end, x:x_end] += pred_tile[:actual_th, :actual_tw] * feather_weights[:actual_th, :actual_tw]
        weight_map[y:y_end, x:x_end] += feather_weights[:actual_th, :actual_tw]
    
    # Normalize by weights for seamless result
    mosaic = np.divide(mosaic, weight_map, out=np.zeros_like(mosaic), where=weight_map!=0)
    
    print("Seamless large-area mosaicking completed!")
    return mosaic

def assess_large_area_quality(prediction, return_detailed_metrics=True):
    """Quality assessment specifically for large-area predictions"""
    metrics = {}
    
    # Basic large-area statistics
    metrics['mean_confidence'] = np.mean(prediction)
    metrics['std_confidence'] = np.std(prediction)
    metrics['coverage_percentage'] = (prediction > 0.5).sum() / prediction.size * 100
    
    # Large-area specific metrics
    binary_pred = (prediction > 0.5).astype(np.uint8)
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(binary_pred)
    
    if num_labels > 1:
        building_areas = stats[1:, cv2.CC_STAT_AREA]
        metrics['num_buildings'] = len(building_areas)
        metrics['avg_building_area'] = np.mean(building_areas)
        metrics['total_building_area'] = np.sum(building_areas)
        metrics['building_density_per_km2'] = len(building_areas) / (prediction.size / 1e6)  # Assuming 10m resolution
    else:
        metrics['num_buildings'] = 0
        metrics['avg_building_area'] = 0
        metrics['total_building_area'] = 0
        metrics['building_density_per_km2'] = 0
    
    return metrics

# Test mosaicking with available predictions
if model_loaded and 'test_predictions' in locals():
    # Create test positions for mosaicking
    test_positions = [{'x': 0, 'y': 0}, {'x': 256, 'y': 0}][:len(test_predictions)]
    
    if len(test_positions) > 0:
        test_mosaic = seamless_large_area_mosaicking(
            test_predictions[:len(test_positions)], 
            test_positions, 
            (1024, 1024)
        )
        
        # Quality assessment
        quality_metrics = assess_large_area_quality(test_mosaic)
        
        print(f"Mosaicking test completed!")
        print(f"Created seamless mosaic of shape {test_mosaic.shape}")
        print(f"Quality metrics:")
        for key, value in quality_metrics.items():
            print(f"  {key}: {value}")
else:
    print("Mosaicking test skipped - no predictions available")

Creating seamless mosaic for large area: 1024 x 1024 pixels


Mosaicking large area: 100%|██████████| 2/2 [00:00<00:00, 450.01it/s]

Seamless large-area mosaicking completed!
Mosaicking test completed!
Created seamless mosaic of shape (1024, 1024)
Quality metrics:
  mean_confidence: 0.0007449278491549194
  std_confidence: 0.0013807985233142972
  coverage_percentage: 0.0
  num_buildings: 0
  avg_building_area: 0
  total_building_area: 0
  building_density_per_km2: 0


In [6]:
# Dask Client Setup for Distributed Processing
def setup_dask_client_for_large_areas(n_workers=None):
    """Setup Dask client optimized for 500km x 500km processing"""
    if n_workers is None:
        n_workers = min(mp.cpu_count(), 4)  # Conservative for large-area processing
    
    try:
        client = Client(
            processes=True, 
            threads_per_worker=2, 
            n_workers=n_workers,
            memory_limit='3GB',  # Conservative memory limit
            dashboard_address=':8787'
        )
        print(f"Dask client for large-area processing started with {n_workers} workers")
        print(f"Dashboard available at: {client.dashboard_link}")
        return client
    except Exception as e:
        print(f"Failed to start Dask client: {e}")
        return None

# Test Dask client setup
dask_client = setup_dask_client_for_large_areas()

if dask_client:
    print("Dask client successfully initialized for distributed processing!")
    print(f"Cluster info: {dask_client}")
else:
    print("Dask client setup failed - will use sequential processing fallback")

Dask client for large-area processing started with 4 workers
Dashboard available at: http://127.0.0.1:8787/status
Dask client successfully initialized for distributed processing!
Cluster info: <Client: 'tcp://127.0.0.1:56007' processes=4 threads=8, memory=11.18 GiB>


In [18]:
# Improved Distributed Processing with Modular Model Loading
def distributed_tile_processing_v2(tiles, model_path, batch_size=10, n_workers=4, dask_client=None):
    """Process tiles using distributed Dask workers with modular model loading"""
    
    # Use the global client if not provided
    if dask_client is None:
        try:
            from dask.distributed import get_client
            dask_client = get_client()
        except Exception:
            print("Error: No Dask client available. Please start a Dask client first.")
            return []
    
    # Import the modular loader in the distributed context
    def process_tile_batch(batch_tiles):
        """Process a batch of tiles in a Dask worker"""
        # Import the modular model loader
        from dask_model_loader import process_tile_with_model
        
        results = []
        for tile_data, tile_info in batch_tiles:
            result = process_tile_with_model(model_path, tile_data, tile_info)
            results.append(result)
        return results
    
    # Split tiles into batches
    tile_batches = []
    for i in range(0, len(tiles), batch_size):
        batch = tiles[i:i+batch_size]
        tile_batches.append(batch)
    
    print(f"Processing {len(tiles)} tiles in {len(tile_batches)} batches using {n_workers} workers")
    
    # Submit batches to Dask workers
    futures = []
    for batch in tile_batches:
        future = dask_client.submit(process_tile_batch, batch)
        futures.append(future)
    
    # Collect results
    all_results = []
    for i, future in enumerate(futures):
        try:
            batch_results = future.result(timeout=300)  # 5 minute timeout per batch
            all_results.extend(batch_results)
            print(f"✓ Completed batch {i+1}/{len(futures)}")
        except Exception as e:
            print(f"✗ Batch {i+1} failed: {e}")
            # Add failed results for this batch
            batch_size_actual = len(tile_batches[i])
            for j in range(batch_size_actual):
                all_results.append({
                    'prediction': np.zeros((512, 512), dtype=np.float32),
                    'tile_info': tile_batches[i][j][1],  # tile_info
                    'status': 'batch_failed'
                })
    
    return all_results


In [19]:
# Test the Improved Distributed Processing
print("Testing improved distributed processing with modular model loading...")

# Use dummy data for testing
test_tiles = []
for i in range(20):  # Test with 20 tiles
    dummy_tile = np.random.randint(0, 255, (512, 512, 3), dtype=np.uint8)
    tile_info = {'row': i//5, 'col': i%5, 'tile_id': f'test_tile_{i}'}
    test_tiles.append((dummy_tile, tile_info))

# Process with improved distributed workers
model_path = r'height_analysis_output\best_height_model.pth'
distributed_results_v2 = distributed_tile_processing_v2(test_tiles, model_path, batch_size=5, n_workers=2)

print(f"\nDistributed processing completed!")
print(f"Processed {len(distributed_results_v2)} tiles")
print(f"Successful predictions: {sum(1 for r in distributed_results_v2 if r['status'] == 'success')}")
print(f"Failed predictions: {sum(1 for r in distributed_results_v2 if r['status'] != 'success')}")

# Verify prediction quality
successful_preds = [r['prediction'] for r in distributed_results_v2 if r['status'] == 'success']
if successful_preds:
    pred_shapes = [p.shape for p in successful_preds]
    pred_ranges = [(p.min(), p.max()) for p in successful_preds[:3]]
    print(f"Prediction shapes: {set(pred_shapes)}")
    print(f"Sample prediction ranges: {pred_ranges}")
    
else:
    print("No successful predictions - investigating...")
    # Show some error details
    error_statuses = [r['status'] for r in distributed_results_v2 if r['status'] != 'success']
    print(f"Error statuses: {set(error_statuses)}")
    
    # Check if modular loader file exists
    import os
    if os.path.exists('dask_model_loader.py'):
        print("Modular loader file exists")
    else:
        print("Modular loader file missing")

Testing improved distributed processing with modular model loading...
Processing 20 tiles in 4 batches using 2 workers
✓ Completed batch 1/4
✓ Completed batch 1/4
✓ Completed batch 2/4
✓ Completed batch 3/4
✓ Completed batch 4/4

Distributed processing completed!
Processed 20 tiles
Successful predictions: 20
Failed predictions: 0
Prediction shapes: {(512, 512)}
Sample prediction ranges: [(np.float32(1.2990328e-06), np.float32(0.03317861)), (np.float32(5.8671765e-07), np.float32(0.059099343)), (np.float32(3.1131524e-06), np.float32(0.022919338))]
✓ Completed batch 2/4
✓ Completed batch 3/4
✓ Completed batch 4/4

Distributed processing completed!
Processed 20 tiles
Successful predictions: 20
Failed predictions: 0
Prediction shapes: {(512, 512)}
Sample prediction ranges: [(np.float32(1.2990328e-06), np.float32(0.03317861)), (np.float32(5.8671765e-07), np.float32(0.059099343)), (np.float32(3.1131524e-06), np.float32(0.022919338))]


In [20]:
# Visualization of Large-Area Processing Results
def visualize_large_area_results(original_shape, prediction, quality_metrics, save_path=None):
    """Visualize large-area processing results"""
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # Original area size visualization
    axes[0, 0].text(0.5, 0.5, f'Large Area Processing\n{original_shape[0]} x {original_shape[1]} pixels\n≈ {original_shape[0]*10/1000:.1f} x {original_shape[1]*10/1000:.1f} km', 
                   ha='center', va='center', fontsize=12, transform=axes[0, 0].transAxes)
    axes[0, 0].set_title('Processing Area Info')
    axes[0, 0].axis('off')
    
    # Prediction visualization
    if prediction is not None:
        axes[0, 1].imshow(prediction, cmap='hot', interpolation='nearest')
        axes[0, 1].set_title('Large Area Prediction')
        axes[0, 1].axis('off')
    
    # Quality metrics
    axes[1, 0].axis('off')
    if quality_metrics:
        metrics_text = f"""
Large Area Quality Metrics:

Buildings Detected: {quality_metrics.get('num_buildings', 0)}
Coverage: {quality_metrics.get('coverage_percentage', 0):.2f}%
Avg Building Area: {quality_metrics.get('avg_building_area', 0):.2f} px²
Building Density: {quality_metrics.get('building_density_per_km2', 0):.2f} per km²
Mean Confidence: {quality_metrics.get('mean_confidence', 0):.3f}
        """
        axes[1, 0].text(0.1, 0.9, metrics_text, transform=axes[1, 0].transAxes,
                        fontsize=10, verticalalignment='top', fontfamily='monospace')
    
    # Processing summary
    axes[1, 1].axis('off')
    axes[1, 1].text(0.1, 0.9,transform=axes[1, 1].transAxes, fontsize=10, verticalalignment='top', fontfamily='monospace')
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Visualization saved to {save_path}")
    
    plt.show()

output_dir = Path("large_area_test_output")
output_dir.mkdir(exist_ok=True, parents=True)

# Visualize results if available
if 'test_mosaic' in locals():
    visualize_large_area_results(
        original_shape=(1024, 1024),
        prediction=test_mosaic,
        quality_metrics=quality_metrics,
        save_path=output_dir / "large_area_results.png"
    )
elif 'test_predictions' in locals() and len(test_predictions) > 0:
    # Create a simple visualization with first prediction
    sample_prediction = test_predictions[0][0] if len(test_predictions[0].shape) == 3 else test_predictions[0]
    sample_metrics = assess_large_area_quality(sample_prediction)
    
    visualize_large_area_results(
        original_shape=sample_prediction.shape,
        prediction=sample_prediction,
        quality_metrics=sample_metrics,
        save_path=output_dir / "sample_results.png"
    )
# Note: Visualization function ready for use when data is available

In [ ]:
# Clean up resources and final summary
if 'dask_client' in locals() and dask_client:
    dask_client.close()
    print("Dask client closed")

# Clear GPU memory
torch.cuda.empty_cache()
gc.collect()


Dask client closed


22

: 